In this document we summarize the methods for wavefield propagation and list the general procedures for their implementation. In particular, we will put special notes on practical problems that requires coders' attention. Scripts will be provided for illustrating the implementation of propagation equations based on the Huygens-Fresnel principle. Based on these discussions, we will demonstrate the construction of a multislice-propagation simulation program. A 3D phantom of size 128x128x128 has already been generated using the XDesign library for this purpose. 

## Overview
All of the wave propagation algorithms discussed here are derived from the well-known Fresnel diffraction integral. They are mathematically equivalent, but differ from each other when it comes to the discrete sampling condition that computers work in. In the text following, we will examine each of these methods in terms of their sampling criteria.

### The convolutional method
The Fresnel diffraction integral can be regarded as the convolution between the original wavefield `u(x, y, 0)` and a Fresnel kernel `h` as 
```
u(x, y, z) = conv(u(x, y, 0), h).
```
It can be easily seen from the diffraction integral that
```
h = exp(j * k * z) / (j * lambda * z) * exp(j * k / (2 * z) * (x**2 + y**2))
```
where `k = 2 * PI / lambda` is the wave number, and `z` is the propagation distance. The invariance of convolution informs us that
```
u(x, y, z) = conv(u(x, y, 0), h)
           = ifft(fft(u(x, y, 0) * fft(h)))
```
The latter form of the equation is of significant computational advantages as it utilize the fast Fourier transform algorithms as an efficiency booster. Depending on how we implement the Fourier transform of `h`, the convolutional methods branches into two mathematically identical but practically quite different algorithms.